# Menu Item to NutrienT (MINT) implementation 

### Text Embedding model 1: RecipeFT

In [ ]:
import numpy as np
import modeling
# sample input
food_data = "Hawaiian Pizza"

# load the model
model = modeling.nutrition_prediction_model()
model.load_weights('data/RecipeFT_weights.bin')

# predict the nutrition density score
prediction = model.predict(food_data)
print("Predicted nutrition density score:", prediction)

### Text Embedding model 2: RecipeBERT

In [2]:
from transformers import pipeline
import torch
import modeling
# sample input
food_data = "Hawaiian Pizza"

try:
    # Use pipeline for feature extraction
    embedding = pipeline('feature-extraction', model='alexdseo/RecipeBERT', framework='pt')

    # Get embeddings
    embeddings = embedding(food_data)
    food_rep = torch.tensor(embeddings).mean(dim=1).squeeze().numpy()
    print(food_rep)

    # predict the nutrition density score
    predicted_nutrition = modeling.nutrition_prediction_model.predict([food_rep])

    print("Predicted Nutritional Facts for 'Hawaiian Pizza':")
    print(f"Calories: {predicted_nutrition[0]['calories']} kcal")
    print(f"Total Fat: {predicted_nutrition[0]['fat']} g")
    print(f"Protein: {predicted_nutrition[0]['protein']} g")
    print(f"Carbohydrates: {predicted_nutrition[0]['carbs']} g")

except Exception as e:
    print(f"An error occurred: {e}")

Some weights of BertModel were not initialized from the model checkpoint at alexdseo/RecipeBERT and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[ 1.27889782e-01  3.48261207e-01  6.86544478e-02 -4.11965698e-02
  2.95388997e-02 -1.68142051e-01 -1.89904287e-01  1.49297535e-01
  3.77540708e-01 -3.56394351e-01  1.14049070e-01 -7.99918324e-02
  2.27789909e-01  1.99802399e-01  4.93500717e-02 -1.71539456e-01
 -2.25293398e-01  3.81586961e-02 -7.70055801e-02 -8.55027586e-02
 -2.95403823e-02 -1.16079405e-01  2.66264617e-01  1.31393179e-01
 -3.88883710e-01 -1.87374160e-01  3.81364301e-02  2.17114776e-01
 -3.33614320e-01  2.13483721e-03  1.31388813e-01 -5.70836291e-02
  1.53143704e-01 -2.93278247e-02  1.75162181e-02 -2.63417631e-01
  1.05568781e-01  4.68118079e-02  3.35588276e-01  2.71861434e-01
  2.44078003e-02 -1.82707205e-01  3.56522977e-01  2.86812633e-01
 -6.61166757e-02 -3.53163928e-01 -2.71564364e-01  1.56412683e-02
 -2.88726300e-01  5.90632483e-03 -2.14421451e-02 -2.41798535e-02
  5.85368127e-02  2.81278759e-01  1.88207537e-01  9.41480845e-02
  1.34096354e-01 -1.17293149e-01  1.72144875e-01  6.69605583e-02
  4.07366097e-01  2.13052

### Nutrition Prediction Model: Using EDAMAM API (https://www.edamam.com/)

In [19]:
import requests

# EDAMAM Food API connection
api_id = 'e1b275a8'
api_key = '54714982ac4e756f3eabea6f47fafa60'
ingredient = "coffee and croissant"
url = f'https://api.edamam.com/api/food-database/v2/parser?app_id={api_id}&app_key={api_key}&ingr={ingredient}'

response = requests.get(url)
data = response.json()
print(data)
unit_table = [('PROCNT', 'g'), ('FIBTG', 'g'), ('VITA_RAE', 'µg'),
                       ('VITC', 'mg'), ('CA', 'mg'), ('FE', 'mg'),
                       ('ENERC_KCAL', 'kcal'), ('SUGAR', 'g'),
                       ('CHOLE', 'mg'), ('FASAT', 'g'), ('NA', 'mg')]

# Initialize nutrient sums
nutrient_sums = {
    'PROCNT': 0, 'FIBTG': 0, 'VITA_RAE': 0, 'VITC': 0, 'CA': 0, 'FE': 0,
    'ENERC_KCAL': 0, 'SUGAR': 0, 'CHOLE': 0, 'FASAT': 0, 'NA': 0
}
nutrient_presence = {key: False for key in nutrient_sums.keys()}

# Sum up all nutrients from all food items
for food in data['hints']:
    nutrients = food['food']['nutrients']
    for nutrient_code, _ in nutrient_sums.items():
        if nutrient_code in nutrients:
            nutrient_sums[nutrient_code] += nutrients[nutrient_code]
            nutrient_presence[nutrient_code] = True

# Print results, distinguish '0' from 'not available'
for nutrient, unit in unit_table:
    if nutrient_presence[nutrient]:
        print(f"{nutrient}: {nutrient_sums[nutrient]} {unit}")
    else:
        print(f"{nutrient}: Not available")


{'text': 'coffee and croissant', 'parsed': [{'food': {'foodId': 'food_ax0a0yxbbe4hx0apiz1tla01s2w7', 'label': 'Coffee', 'knownAs': 'coffee', 'nutrients': {'ENERC_KCAL': 1.0, 'PROCNT': 0.12, 'FAT': 0.02, 'CHOCDF': 0.0, 'FIBTG': 0.0}, 'category': 'Generic foods', 'categoryLabel': 'food', 'image': 'https://www.edamam.com/food-img/ee9/ee9566349cb84dfd9ddac1fdf8cbc907.jpg'}}, {'food': {'foodId': 'food_b517334aiij8xjave2iu9b75zron', 'label': 'Croissant', 'knownAs': 'croissants', 'nutrients': {'ENERC_KCAL': 406.0, 'PROCNT': 8.2, 'FAT': 21.0, 'CHOCDF': 45.8, 'FIBTG': 2.6}, 'category': 'Generic foods', 'categoryLabel': 'food', 'image': 'https://www.edamam.com/food-img/590/590b8e57dd922af23adc3b4578a9033a.gif'}}], 'hints': [{'food': {'foodId': 'food_ax0a0yxbbe4hx0apiz1tla01s2w7', 'label': 'Coffee', 'knownAs': 'coffee', 'nutrients': {'ENERC_KCAL': 1.0, 'PROCNT': 0.12, 'FAT': 0.02, 'CHOCDF': 0.0, 'FIBTG': 0.0}, 'category': 'Generic foods', 'categoryLabel': 'food', 'image': 'https://www.edamam.com/

In [5]:
# EDAMAM Nutrition API connection
api_id = '7e35dd62'
api_key = '2b51dcc40f369ed4ae859021a0508f00'
url = 'https://api.edamam.com/api/nutrition-details?app_id={}&app_key={}'.format(api_id, api_key)

headers = {
    'Content-Type': 'application/json'
}

data = {
    'ingr': ['1 large apple', '1 cup of green tea']
}

response = requests.post(url, headers=headers, json=data)
print(response.json())

{'uri': 'http://www.edamam.com/ontologies/edamam.owl#recipe_a3232a1869ad4d5c9d05d46cb27a9233', 'yield': 2.0, 'calories': 118, 'totalCO2Emissions': 515.2810000000001, 'co2EmissionsClass': 'C', 'totalWeight': 468.0, 'dietLabels': ['LOW_FAT', 'LOW_SODIUM'], 'healthLabels': ['FAT_FREE', 'LOW_FAT_ABS', 'LOW_POTASSIUM', 'KIDNEY_FRIENDLY', 'VEGAN', 'VEGETARIAN', 'PESCATARIAN', 'SPECIFIC_CARBS', 'DAIRY_FREE', 'GLUTEN_FREE', 'WHEAT_FREE', 'EGG_FREE', 'MILK_FREE', 'PEANUT_FREE', 'TREE_NUT_FREE', 'SOY_FREE', 'FISH_FREE', 'SHELLFISH_FREE', 'PORK_FREE', 'RED_MEAT_FREE', 'CRUSTACEAN_FREE', 'CELERY_FREE', 'MUSTARD_FREE', 'SESAME_FREE', 'LUPINE_FREE', 'MOLLUSK_FREE', 'ALCOHOL_FREE', 'NO_OIL_ADDED', 'NO_SUGAR_ADDED', 'SULPHITE_FREE', 'KOSHER'], 'cautions': ['SULFITES'], 'totalNutrients': {'ENERC_KCAL': {'label': 'Energy', 'quantity': 118.41, 'unit': 'kcal'}, 'FAT': {'label': 'Total lipid (fat)', 'quantity': 0.37910000000000005, 'unit': 'g'}, 'FASAT': {'label': 'Fatty acids, total saturated', 'quantity'